### EDA

Görs generellt på träningsdatan endast för att undvika dataläckage. 

Var försiktig med att göra åtgärder innan modellering. 

Identifiera mönster - Behöver vi göra en modell? 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cars = pd.read_csv('car_price_dataset.csv', sep=';')

1. Förstå datasetet

In [ ]:
print(cars.shape) # Antalet rader och kolumner
print(cars.info()) # Datatyper
print(cars.head()) # Se exempel

2. Deskriptiv statistik

* Förstå spridning. Medelpris = Typiskt värde. Variation/standardavvikelse = Hur mycket skiljer sig värdena åt. Extremvärden = Misstänkta outliers. 
* Upptäcka fel och konstiga värden. Tex negativa värden eller outliers. Kategorier med få observationer kanske inte har tillräckligt med data för analys och måste slås ihop med en annan kategori. 
* Hjälp för att tolka modellens prediktioner senare. 

In [ ]:
print(cars.describe())
print("\nKategoriska variabler:")
print("Brand:\n", cars['Brand'].value_counts())
print("\nFuel_Type:\n", cars['Fuel_Type'].value_counts())
print("\nTransmission:\n", cars['Transmission'].value_counts())

3. Hitta saknade värden

In [ ]:
print(cars.isnull().sum())

4. Visualisera data

* Histogram --> fördelning
* Boxplot --> outliers
* Scatterplot --> samband

In [ ]:
plt.figure(figsize=(8,5))
plt.hist(cars['Price'], bins=20, color='skyblue', edgecolor='black')
plt.title("Histogram över bilpriser")
plt.xlabel("Pris")
plt.ylabel("Antal bilar")
plt.show()

# Visar på form på fördelning: Symmetrisk (normalfördelad), skev åt höger, skev åt vänster

Hur långt från medelvärdet värdena ligger --> stor eller liten spridning. 

Vissa modeller är mycket känsliga för stor spridning.

In [ ]:
data = np.random.normal(loc=0, scale=1, size=1000)
plt.hist(data, bins=30, density=True)
plt.title("Exempel på normalfördelning")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
brands = cars['Brand'].unique()
data_to_plot = [cars[cars['Brand']==brand]['Price'] for brand in brands]

plt.boxplot(data_to_plot, labels=brands, patch_artist=True)
plt.title("Boxplot av pris per bilmärke")
plt.xlabel("Märke")
plt.ylabel("Pris")
plt.xticks(rotation=45)
plt.show()
# Identifiera outliers som är relativa till gruppen, inte hela datan.

In [ ]:
plt.figure(figsize=(8,5))
plt.boxplot(cars['Mileage'], patch_artist=True)
plt.title("Boxplot av Mileage")
plt.ylabel("Mileage (km)")
plt.show()

# Identifiera allmänna outliers. 

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(cars['Mileage'], cars['Price'], alpha=0.6, color='green')
plt.title("Scatterplot: Mileage vs Price")
plt.xlabel("Mileage")
plt.ylabel("Pris")
plt.show()

#### Normalfördelning 

Symmetrisk spridning runt medelvärdet med färre extremvärden.

Kan påverkar vissa modeller och andra inte om variabler inte är normalfördelade. 

Om residualerna i en regression är normalfördelade, kan vi korrekt ange sannolikheten för olika prediktioner och konfidensintervall.

Åtgärd: 

* Transformation av variabler kan vara nödvändig. 
* Ta bort outliers. 
* Prova annan modell som inte påverkas av skevhet. 

#### Outliers

Ett värde som avviker kraftigt från resten av datan. Kan påverka vår analys krafigt. Kan vara datafel eller naturliga variationer. 

Åtgärd: 

* Ta bort (om felaktiga tex)
* Använd en mer robust modell (kan dock bli onödigt komplex modell även om den fungerar bra med outliers i data)
* Transformera variabler för att minska påverkan
* Ingenting

#### icke-linjära samband

Kanske behöver vi en icke-linjär modell? 

#### Skalning

När behöver data skalas/normaliseras? 

* Om variabler mäts i helt olika enheter och skiljer sig mycket ifrån varandra eller har stor spridning
* När du ska använda en avståndsbaserad modell (Trädmodeller behöver normalt inte skalas)

5. Korrelationsanalys

Korrelationsanalys är viktigt för att förstå hur variabler samvarierar och vilka som kan vara viktiga i en modellering. 

Tittar på styrkan i ett **linjärt** samband. 

Förstå relationer till målvariablen. 
Ger tidiga hypoteser:
* “Horsepower verkar påverka priset starkt”
* “Age har svag relation → kanske mindre viktig”

* Variabler som inte alls korrelerar med målvariabeln → kanske inte användbara
* Variabler som är mycket korrelerade → risk för multikollinearitet. = Två (eller flera) förklarande variabler ör starkt korrelerade med varandra i en modell, alltså modellen får flera variabler som säger samma sak (tex enging size och horsepower). Svårt att tolka vilken variabel som faktiskt påverkar målvariabeln. Kan ge en bra prediktion i vissa fall, men tolkning kan vara mycket svårt. 

Åtgärd:

* Ta bort en av de korrelerade variablerna. 
* Slå ihop korrelerade variabler. 
* Används regularisering
* Dimensionsreducering (om tolkning inte är viktigt)
* Använd en annan modell.

In [ ]:
# Korrelation mellan numeriska variabler och Price
numeric_cols = ['Year', 'Engine_Size', 'Mileage', 'Doors', 'Owner_Count', 'Price']
print(cars[numeric_cols].corr()['Price'].sort_values(ascending=False))

In [ ]:
# Korrelations heatmap

df_numeric = cars.select_dtypes(include='number')
corr = df_numeric.corr()

fig, ax = plt.subplots(figsize=(10, 8))

# Rita heatmap
cax = ax.imshow(corr, cmap='coolwarm')

# Lägg till färgskala
fig.colorbar(cax)

# Sätt axel-etiketter
ax.set_xticks(np.arange(len(corr.columns)))
ax.set_yticks(np.arange(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=45, ha='right')
ax.set_yticklabels(corr.columns)

# Visa korrelationsvärden i rutorna
for i in range(len(corr.columns)):
    for j in range(len(corr.columns)):
        ax.text(j, i, f"{corr.iloc[i, j]:.2f}",
                ha="center", va="center", color="black")

ax.set_title("Correlation Heatmap")
plt.tight_layout()
plt.show()

# Utforska gärna bibliotektet Seaborn för snyggare grafer med mindre kod. 

In [ ]:
# alternativ till ovan

# Alternativ: Enklare heatmap med Seaborn (samma resultat)
plt.figure(figsize=(10,8))
sns.heatmap(df_numeric.corr(), annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title("Korrelationsmatris (Seaborn)")
plt.tight_layout()
plt.show()


Heatmapen kan hjälpa dig med:

* Feature selection - Behåll variabler med stark korrelation med målvariabeln
* Identifiera redundanta variabler - Om två features är starkt korrelerade med varandra → välj en

Tänk på att heatmapen ger en indikation. Bara för att Horsepower har en hög korrelation betyder inte det att det är den som  "orsakar" priset. Icke-linjära samband syns inte alltid. Variabel kan vara viktig trots låg korrelation. 